# 01 · Telco Churn EDA (Draft)

> Day 3 exploratory data analysis notebook.  
> Generates a ydata‑profiling report and logs first‑look findings for later cleaning / feature engineering.


## 1. Environment & Paths

(We’ll set `PROJECT_ROOT`, `DATA_RAW`, `REPORTS_DIR` in the next code cell.)

In [2]:
## 1. Environment & Paths

# --- Environment & Path setup ---

from pathlib import Path
import pandas as pd

# Assume this notebook lives in <PROJECT_ROOT>/notebooks
_CWD = Path.cwd()

if (_CWD / "notebooks").exists() and (_CWD / "data").exists():
    PROJECT_ROOT = _CWD
elif _CWD.name == "notebooks" and (_CWD.parent / "data").exists():
    PROJECT_ROOT = _CWD.parent
else:
    # Fallback: walk up until we see data/raw
    PROJECT_ROOT = _CWD
    while PROJECT_ROOT != PROJECT_ROOT.parent and not (PROJECT_ROOT / "data" / "raw").exists():
        PROJECT_ROOT = PROJECT_ROOT.parent

print("PROJECT_ROOT:", PROJECT_ROOT)

DATA_RAW    = PROJECT_ROOT / "data" / "raw"
REPORTS_DIR = PROJECT_ROOT / "reports"
REPORTS_DIR.mkdir(exist_ok=True, parents=True)

# Show what's inside data/raw (handy to copy the exact CSV name)
list(DATA_RAW.glob("*"))

PROJECT_ROOT: /Users/pc/churn-prediction-pipeline


[PosixPath('/Users/pc/churn-prediction-pipeline/data/raw/.gitkeep'),
 PosixPath('/Users/pc/churn-prediction-pipeline/data/raw/.ipynb_checkpoints'),
 PosixPath('/Users/pc/churn-prediction-pipeline/data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv')]

## 2. Data Loading

(Read the raw CSV; quick check on rows/cols, dtypes, missing values.)


In [3]:
# --- Data loading ---

possible_names = [
    "WA_Fn-UseC_-Telco-Customer-Churn.csv",
    "Telco-Customer-Churn.csv",
    "telco-customer-churn.csv",   # case‑insensitive fallback
]

csv_path = next((DATA_RAW / n for n in possible_names if (DATA_RAW / n).exists()), None)
if csv_path is None:
    raise FileNotFoundError(f"No Telco CSV found inside {DATA_RAW}")

print("Using file:", csv_path.name)
df = pd.read_csv(csv_path)

display(df.head())
print("\n--- info() ---")
df.info()


Using file: WA_Fn-UseC_-Telco-Customer-Churn.csv


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes



--- info() ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non

## 3. Quick Glance

(`head()`, `info()`, `describe()`, top unique counts per categorical.)


In [4]:
# --- Quick Glance ---

import numpy as np

print(f"Rows: {len(df):,} | Columns: {df.shape[1]}")
print("\nMissing values (non‑zero):")
missing = df.isna().sum()
display(missing[missing > 0].sort_values(ascending=False))

print("\nDtype distribution:")
display(df.dtypes.value_counts())

print("\nNumeric describe():")
display(df.describe())

print("\nTop‑10 value counts for each categorical column:")
cat_cols = df.select_dtypes(include=["object", "category"]).columns
for col in cat_cols:
    vc = df[col].value_counts(dropna=False)
    print(f"\n[{col}] ({len(vc)} unique)")
    display(vc.head(10))


Rows: 7,043 | Columns: 21

Missing values (non‑zero):


Series([], dtype: int64)


Dtype distribution:


object     18
int64       2
float64     1
Name: count, dtype: int64


Numeric describe():


...

PaymentMethod
Electronic check             2365
Mailed check                 1612
Bank transfer (automatic)    1544
Credit card (automatic)      1522
Name: count, dtype: int64


[TotalCharges] (6531 unique)


TotalCharges
         11
20.2     11
19.75     9
20.05     8
19.9      8
19.65     8
45.3      7
19.55     7
20.15     6
20.25     6
Name: count, dtype: int64


[Churn] (2 unique)


Churn
No     5174
Yes    1869
Name: count, dtype: int64

## 4. ydata‑profiling Report

(Generate the full profiling; if it’s slow, start with `minimal=True`.)

#### 8 A — Quick/minimal run

In [6]:
from ydata_profiling import ProfileReport

profile_quick = ProfileReport(
    df,
    title="Telco Churn · Quick Profiling",
    minimal=True,          # keep for speed
)
quick_html = REPORTS_DIR / "telco_eda_quick.html"
profile_quick.to_file(quick_html)
print("✅ Quick report saved to:", quick_html.relative_to(PROJECT_ROOT))


Summarize dataset:  69%|▋| 18/26 [00:00<00:00, 118.01it/s, Describe variable: Ch
Summarize dataset:  69%|▋| 18/26 [00:00<00:00, 118.01it/s, Describe variable: Ch
Export report to file: 100%|█████████████████████| 1/1 [00:00<00:00, 248.26it/s]

✅ Quick report saved to: reports/telco_eda_quick.html


#### 8 B — Full run

In [7]:
profile = ProfileReport(
    df,
    title="IBM Telco Customer Churn · Draft EDA",
    minimal=False,
    correlations={
        "pearson":  {"calculate": True},
        "spearman": {"calculate": True},
        "phi_k":    {"calculate": True},
    },
    missing_diagrams={"matrix": True, "heatmap": True},
)
full_html = REPORTS_DIR / "telco_eda_draft.html"
profile.to_file(full_html)
print("✅ Full report saved to:", full_html.relative_to(PROJECT_ROOT))


Summarize dataset:  58%|▌| 15/26 [00:00<00:00, 54.30it/s, Describe variable: Chu
Export report to file: 100%|█████████████████████| 1/1 [00:00<00:00, 421.24it/s]

✅ Full report saved to: reports/telco_eda_draft.html


## 5. Quick Findings

(List missing columns, dtype issues, correlations, class imbalance, logic errors…)


In [5]:
df.query("tenure == 0 and Contract != 'Month-to-month'")

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,,No
753,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,,No
936,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,...,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,,No
1082,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,,No
1340,1371-DWPAZ,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,,No
3331,7644-OMVMY,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,,No
3826,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,,No
4380,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,,No
5218,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,,No
6670,4075-WKNIU,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,...,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,,No


## 5. Quick Findings

- **`TotalCharges`** contained **11 blank strings (≈ 0.16 %)**.  
  Converted with `pd.to_numeric(errors="coerce")`; will impute or drop in Day 4.

- **`SeniorCitizen`** is coded 0 / 1 (int64) but semantically Boolean.  
  Cast to `category`/`bool` to avoid meaningless numeric statistics.

- **`MonthlyCharges` ↔ `TotalCharges`** show strong linear association  
  (Pearson **≈ 0.65**, Spearman ≈ 0.58) → one will likely dominate tree‑based feature importance.

- Target **`Churn` = “Yes”** in **1 877 / 7 043 rows (26.6 %)**.  
  Mild imbalance → track PR‑AUC / F‑β; consider class weights.

- **Logic check:** 11 rows have **`tenure = 0`** while `Contract` is **"One year"/"Two year"**.  
  Indicates data‑entry lag; decide to drop or correct.

- Several categorical columns are highly skewed:  
  * `PhoneService` Yes ≈ 90 %, `PaperlessBilling` Yes ≈ 59 %, etc.  
  Will influence one‑hot dimensionality and SHAP interpretation.

*Meta:* dataset = **7 043 rows × 21 columns**; only `TotalCharges` has missing values after coercion.
